In [10]:
import json
with open(f'data/landing/fantasy/2022/races.json', 'r') as f:
    file = json.load(f)

In [12]:
from pyarrow.parquet import read_metadata as parquet_metadata

meta = parquet_metadata(f'data/bronze/fantasy/races.parquet').to_dict()
rows = meta['num_rows']
cols = meta['num_columns']

{'created_by': 'Polars',
 'num_columns': 5,
 'num_rows': 44,
 'num_row_groups': 1,
 'row_groups': [{'num_columns': 5,
   'num_rows': 44,
   'total_byte_size': 1373,
   'columns': [{'file_offset': 170,
     'file_path': '',
     'physical_type': 'BYTE_ARRAY',
     'num_values': 44,
     'path_in_schema': 'country_code',
     'is_stats_set': False,
     'statistics': None,
     'compression': 'ZSTD',
     'encodings': ('PLAIN', 'RLE'),
     'has_dictionary_page': False,
     'dictionary_page_offset': None,
     'data_page_offset': 4,
     'total_compressed_size': 166,
     'total_uncompressed_size': 339},
    {'file_offset': 305,
     'file_path': '',
     'physical_type': 'BYTE_ARRAY',
     'num_values': 44,
     'path_in_schema': 'event_format',
     'is_stats_set': False,
     'statistics': None,
     'compression': 'ZSTD',
     'encodings': ('PLAIN', 'RLE'),
     'has_dictionary_page': False,
     'dictionary_page_offset': None,
     'data_page_offset': 204,
     'total_compressed_si

In [14]:
len(file['races'])

22